<a href="https://colab.research.google.com/github/faizankshaikh/ForaGym/blob/main/notebooks/ExampleForagym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install required libraries

In [1]:
# !git clone --quiet https://github.com/faizankshaikh/ForaGym.git
# %cd ForaGym/
# !pip install -q -e .
# !pip install -q gymnasium stable-baselines3==2.0.0a5 shap
# !pip install -q --no-deps imitation

/content/ForaGym


# 2. Create Gym environment

In [7]:
# import required libs and modules
import pandas as pd
import gymnasium as gym

In [6]:
# initiatlize gym env
env_verbose = gym.make(
    "foragym:foragym/ForaGym-v1",
    render_mode="human"
)

In [10]:
# visualize transition matrix
num_states = env_verbose.nS
num_actions = env_verbose.nA

df = []
for state in range(num_states):
    for action in range(num_actions):
        for items in env_verbose.P[state][action]:
            try:
                p, new_state, reward, done = items
                days_left, life_point, forest_type = env_verbose.decode(state)
                days_left_next, life_point_next, forest_type_next = env_verbose.decode(new_state)
                df.append([
                    state, 
                    days_left, 
                    life_point, 
                    forest_type, 
                    env_verbose.action_dict[action],
                    days_left_next,
                    life_point_next,
                    forest_type_next,
                    round(p, 3),
                    reward,
                    done
                ])
            except:
                pass

df = pd.DataFrame(df, columns=[
    "id_state", 
    "days_left", 
    "life_points_left_left", 
    "id_forest", 
    "action", 
    "days_left_next", 
    "life_points_left_next",
    "id_forest_next",
    "prob_transition", 
    "reward", 
    "terminated"
])
df.head()

,id_state,days_left,life_points_left_left,id_forest,action,days_left_next,life_points_left_next,id_forest_next,prob_transition,reward,terminated
0,576,1,1,0,wait,0.0,0.0,0.0,1.000,-1,True
1,576,1,1,0,forage,0.0,0.0,0.0,0.333,-1,True
2,576,1,1,0,forage,0.0,0.0,0.0,0.125,-1,True
3,576,1,1,0,forage,0.0,2.0,0.0,0.167,0,True
4,576,1,1,0,forage,0.0,2.0,0.0,0.125,0,True


# 3. Solve the foraging task

In [11]:
# run algorithm on sample episodes
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    done = False
    print("Initial state:-")
    obs = env_verbose.reset(seed=42)
    print()

    while not done:
        action = env_verbose.action_space.sample()
        print(f"Action to take: {env_verbose.action_dict[action]}")
        print()
        obs, reward, done, _, info = env_verbose.step(action)
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Forest Quality for the left environment: 0.50
--Threat Encounter probability for the left environment: 0.00
--Nutritional Quality for the left environment: 2.00
--Forest Quality for the right environment: 0.57
--Threat Encounter probability for the right environment: 0.29
--Nutritional Quality for the right environment: 2.00
----------

--Days left: 7
--Current life: 6
--Current Forest Quality: 0.57
--Current Threat Encounter probability: 0.29
--Current Nutritional Quality: 2.00

Action to take: forage

--Consequence: Right environment / Forage failed / No threat encountered
--Reward?: 0
--Episode done?: False
--Days left: 6
--Current life: 4
--Current Forest Quality: 0.50
--Current Threat Encounter probability: 0.00
--Current Nutritional Quality: 2.00

Action to take: wait

--Consequence: Waited
--Reward?: 0
--Episode done?: False
--Days left: 5
--Current life: 3
--Current Forest Quality: 0.57
--Current Threat Encounter probability: 0.29
--Current Nutritio